在pytorch中,一般只对叶子节点进行梯度计算;而对于非叶子节点，则没有显式地去保留其中间计算过程中的梯度.

In [16]:
import torch
from torch.autograd import Variable

In [41]:
x = torch.tensor(3., requires_grad=True)
y = x ** 2
# y.retain_grad()
z = 4 * y
y_grad = 0.
 
 
# 为了读取模型中间参数变量的梯度而定义的辅助函数
def extract(g):
    global y_grad
    y_grad = g
 
 
y.register_hook(extract)
z.backward()
 
print(x.grad)   # tensor(24.)
print(y_grad)   # tensor(4.)

tensor(24.)
tensor(4.)


**Tensor.retain_grad()**:显式地保存非叶节点的梯度，代价是会增加显存的消耗，而用`hook`函数的方法则是在反向计算时直接打印，因此不会增加显存消耗，但是使用起来`retain_grad()`要比`hook`函数方便一些。

In [42]:
x = Variable(torch.ones(2, 2), requires_grad=True)
y = x + 2
y.retain_grad()
z = y * y * 3
out = z.mean()
out.backward()
print(y.grad)
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


**hook**

In [44]:
grads = {}
def save_grad(name):
    def hook(grad):
        grads[name] = grad
    return hook

x = Variable(torch.randn(1,1), requires_grad=True)
y = 3*x
z = y**2

# In here, save_grad('y') returns a hook (a function) that keeps 'y' as name
y.register_hook(save_grad('y'))
z.register_hook(save_grad('z'))
z.backward()

print(grads['y'])
print(grads['z'])

tensor([[1.0705]])
tensor([[1.]])


In [45]:
import torch
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class MyMul(nn.Module):
    def forward(self, input):
        out = input * 2
        return out

class MyMean(nn.Module):            # 自定义除法module
    def forward(self, input):
        out = input/4
        return out

def tensor_hook(grad):
    print('tensor hook')
    print('grad:', grad)
    return grad

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.f1 = nn.Linear(4, 1, bias=True)    
        self.f2 = MyMean()
        self.weight_init()

    def forward(self, input):
        self.input = input
        output = self.f1(input)       # 先进行运算1，后进行运算2
        output = self.f2(output)      
        return output

    def weight_init(self):
        self.f1.weight.data.fill_(8.0)    # 这里设置Linear的权重为8
        self.f1.bias.data.fill_(2.0)      # 这里设置Linear的bias为2

    def my_hook(self, module, grad_input, grad_output):
        print('doing my_hook')
        print('original grad:', grad_input)
        print('original outgrad:', grad_output)
        # grad_input = grad_input[0]*self.input   # 这里把hook函数内对grad_input的操作进行了注释，
        # grad_input = tuple([grad_input])        # 返回的grad_input必须是tuple，所以我们进行了tuple包装。
        # print('now grad:', grad_input)        

        return grad_input

if __name__ == '__main__':

    input = torch.tensor([1, 2, 3, 4], dtype=torch.float32, requires_grad=True).to(device)

    net = MyNet()
    net.to(device)

    net.register_backward_hook(net.my_hook)   # 这两个hook函数一定要result = net(input)执行前执行，因为hook函数实在forward的时候进行绑定的
    input.register_hook(tensor_hook)
    result = net(input)

    print('result =', result)

    result.backward()

    print('input.grad:', input.grad)
    for param in net.parameters():
        print('{}:grad->{}'.format(param, param.grad))

result = tensor([20.5000], device='cuda:0', grad_fn=<DivBackward0>)
doing my_hook
original grad: (tensor([0.2500], device='cuda:0'), None)
original outgrad: (tensor([1.], device='cuda:0'),)
tensor hook
grad: tensor([2., 2., 2., 2.], device='cuda:0')
input.grad: None
Parameter containing:
tensor([[8., 8., 8., 8.]], device='cuda:0', requires_grad=True):grad->tensor([[0.2500, 0.5000, 0.7500, 1.0000]], device='cuda:0')
Parameter containing:
tensor([2.], device='cuda:0', requires_grad=True):grad->tensor([0.2500], device='cuda:0')


E:\Anaconda3\lib\site-packages\torch\nn\modules\module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
<ipython-input-45-8b271c331ca8>:63: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  print('input.grad:', input.grad)
